# LCA with foreground

This notebook demonstrates how to include foregrounds system into the analysis(concatenate foreground system with background system). It covers two sections:
1) Foreground system only have one activity
2) Foreground system only have two activities

### Preparation

1. Before use the library, you have to direct to the right brightway project and make sure you have the database.

In [1]:
import bw2data as bd

# bd.projects.set_current("advlca25")
bd.projects.set_current("advlca23")  # use old cf values to reproduce the old lca results.
bd.databases

Databases dictionary with 11 object(s):
	ALIGNED-biob-prod-dummy
	SAtestdb
	biosphere3
	carbon fiber
	ecoinvent 3.9.1 conseq
	exldb
	gsa_db
	sa_db
	sa_db_param
	testbiosphere
	testdb

2. Import all needed libraries

In [2]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_bamboo.background_importer import *
from bw_bamboo.foreground_importer import *
from bw_bamboo.datapackage_builder import *
from bw_bamboo.uncertainty_handler import *
from bw_bamboo.lca_wrapper import *

3. Define some constants for this notebook

In [ ]:
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - biogenic - air", 
        "CO2 - waste - fossil - air",
        "CO2 - agriculture - peat decay - air", 
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
FG_MAPPING_FILE = os.path.join(os.getcwd(), "data", "EXIOBASE-ecoinvent-bio-bw-GHG.csv")
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")
EMISSION_CODE_FILE = os.path.join(os.getcwd(), "data/EXIOBASE-ecoinvent-bio-bw-GHG.csv")
FROEGROUND_1 = os.path.join(os.getcwd(), "data/20250110_foreground_system_small.csv")
# FROEGROUND_1 = os.path.join(os.getcwd(), "data/fg_exiobase_aggregated_1col.csv")
FROEGROUND_2 = os.path.join(os.getcwd(), "data/fg_exiobase_aggregated_2col.csv")


## Section 1: Add foreground system with 1 column

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.form_tech_matrix(raw_tech)

2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.form_bio_matrix(bio_df, GHG)

3. Get characterization factor matrix

In [ ]:
# cf_matrix = bg_importer.form_cf_matrix("ecoinvent-3.11-biosphere", METHOD, FG_MAPPING_FILE, GHG)
# cf_matrix.diagonal()  # print the diagnal for checking

# since 2 of the emissions we chose has no cf value in IPCC2013 method, we assign the values so that we can use.
cf_matrix = [1., 1., 1., 1., 1., 1., 27.0, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 29.8, 273., 273., 25200.]
cf_matrix = np.diagflat(cf_matrix)

4. Check out the foreground data

In [7]:
fg_tech_df = pd.read_table(FROEGROUND_1, sep=',')
fg_tech_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,u,Exchange negative
0,column_1,EU28-Agriculture-Forestry-Fishing,technosphere,0.000000,0,NaN,NaN
1,column_1,EU28-Energy,technosphere,NaN,0,NaN,NaN
2,column_1,EU28-Natural gas and services related to natur...,technosphere,NaN,0,NaN,NaN
3,column_1,EU28-Industry,technosphere,0.000001,2,3.330005,False
4,column_1,EU28-Motor Gasoline,technosphere,-0.000016,2,1.540000,True


5. Get activities in foreground system.

In [8]:
fg_activities_1 = get_fg_activities(FROEGROUND_1, ",")
fg_activities_1

['column_1']

6. Get activities in background system.

In [9]:
bg_activities_1 = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities_1[:5] # only list the first five activities here

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline']

7. Get foreground system matrices

In [10]:
fg_importer = ForegroundImporter()

fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_tech_df, GHG, fg_activities_1, bg_activities_1)

8. Concatenate foreground system with background system

In [11]:
full_tech_matrix_1, full_bio_matrix_1 = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

# to compare the matrix before and after concatenate with foreground system
print(f"Technosphere shape change to {full_tech_matrix_1.shape} from {tech_matrix.shape}")
print(f"Biosphere shape change to {full_bio_matrix_1.shape} from {bio_matrix.shape}")

Technosphere shape change to (77, 77) from (76, 76)
Biosphere shape change to (20, 77) from (20, 76)


9. Build datapackage

In [12]:
dp_builder = DatapackageBuilder()
datapackage_data_1 = dp_builder.prepare_dp_matrix(full_tech_matrix_1, full_bio_matrix_1, cf_matrix)
datapackage_1 = dp_builder.prepare_datapackage(datapackage_data_1)

10. Run static simulation

In [13]:
mysector_1 = 69  # RoW-Services

activities = fg_activities_1 + bg_activities_1

lca = bc.LCA(
            demand={mysector_1 : 1},
            data_objs=[datapackage_1],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector_1]}")


Brightway calculated lca score: (291685.0667897694, 'RoW-Services')


## Section 2: Add foreground system with 2 columns

1. Check out the foreground data

In [14]:
fg_tech_df_2 = pd.read_table(FROEGROUND_2, sep=',')
fg_tech_df_2.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,GSD,Exchange negative
0,column_1,EU28-Electricity by biomass and waste,technosphere,-2.340000e-07,2,2.674795,True
1,column_1,EU28-Electricity by solar photovoltaic,technosphere,-2.010000e-07,2,2.674795,True
2,column_1,EU28-Electricity by solar thermal,technosphere,0.000000e+00,1,0.000000,False
3,column_1,"EU28-Electricity by tide, wave, ocean",technosphere,-6.780000e-09,2,2.674795,True
4,column_1,EU28-Electricity by Geothermal,technosphere,-9.350000e-10,2,2.674795,True


2. Get activities in foreground system.

In [15]:
fg_activities_2 = get_fg_activities(FROEGROUND_2, ",")
fg_activities_2

['column_1', 'column_2']

3. Get activities in background system.

In [16]:
bg_activities_2 = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities_2[:5] # only list the first five activities here

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline']

4. Get foreground system matrices

In [17]:
fg_importer = ForegroundImporter()

fgbg_2, fgfg_2, bgfg_2, bifg_2 = fg_importer.extend_matrix(fg_tech_df_2, GHG, fg_activities_2, bg_activities_2)

5. Concatenate foreground system with background system

In [18]:
full_tech_matrix_2, full_bio_matrix_2 = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg_2, fgfg_2, bgfg_2, bifg_2)

# to compare the matrix before and after concatenate with foreground system
print(f"Technosphere shape change from {full_tech_matrix_2.shape} to {tech_matrix.shape}")
print(f"Biosphere shape change from {full_bio_matrix_2.shape} to {bio_matrix.shape}")

Technosphere shape change from (78, 78) to (76, 76)
Biosphere shape change from (20, 78) to (20, 76)


6. Build datapackage

In [19]:
dp_builder = DatapackageBuilder()
datapackage_data_2 = dp_builder.prepare_dp_matrix(full_tech_matrix_2, full_bio_matrix_2, cf_matrix)
datapackage_2 = dp_builder.prepare_datapackage(datapackage_data_2)

7. Run static simulation

In [20]:
mysector_2 = 70  # RoW-Services

activities = fg_activities_2 + bg_activities_2

lca = bc.LCA(
            demand={mysector_2 : 1},
            data_objs=[datapackage_2],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector_2]}")


Brightway calculated lca score: (291685.0667897694, 'RoW-Services')


## Run the simulation manually to compare

1. With both foreground system (1 col) and background system

In [21]:
from bw_bamboo.lca_wrapper import *

np.fill_diagonal(full_tech_matrix_1, -full_tech_matrix_1.diagonal())
full_tech_matrix_1 = -full_tech_matrix_1

rca = LCAWrapper()
rca.manual_lca(full_tech_matrix_1, full_bio_matrix_1, cf_matrix, mysector_1)

291685.06678976933

2. With both foreground system (2 cols) and background system

In [22]:
from bw_bamboo.lca_wrapper import *

np.fill_diagonal(full_tech_matrix_2, -full_tech_matrix_2.diagonal())
full_tech_matrix_2 = -full_tech_matrix_2

rca = LCAWrapper()
rca.manual_lca(full_tech_matrix_2, full_bio_matrix_2, cf_matrix, mysector_2)

291685.0667897693